In [ ]:
TRAIN_PATH = ""
VAL_PATH = ""

In [ ]:
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import itertools
import numpy as np

In [ ]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),activation = 'relu', input_shape = (224,224,3)))

model.add(Conv2D(64,kernel_size=(3,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy, optimizer = 'adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
       )  

test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [ ]:

train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)



In [ ]:
train_generator.class_indices
# class_weight = {0 : 0.4,1: 0.6}


In [ ]:
validation_generator = test_datagen.flow_from_directory(
    VAL_PATH,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary',
    shuffle=False
)

In [ ]:
num_epochs = 10
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=7,
    epochs = num_epochs,
    validation_data = validation_generator,
    validation_steps=2,
)

In [ ]:
preds = model.predict_classes(validation_generator)
y_pred = [int(p) for p in preds]
y_true = validation_generator.labels

In [ ]:
str2idx = validation_generator.class_indices
idx2str = {0:"Covid", 1:"Normal"}

In [ ]:
def num2str(lst):
    return [idx2str[i] for i in lst]

In [ ]:
y_pred_str = num2str(y_pred)
y_true_str = num2str(y_true)

In [ ]:
cm = confusion_matrix(y_true=y_true_str, y_pred=y_pred_str)
classes = validation_generator.class_indices

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Greys):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
plot_confusion_matrix(cm,classes)

In [ ]:
print(classification_report(y_true=y_true, y_pred=y_pred, target_names=validation_generator.class_indices))
print(hist.history.keys())

In [ ]:
print(hist.history.keys())

plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Learning curves')
plt.ylabel('accuracy / loss')
plt.xlabel('epoch')
plt.legend(['train_accuracy', 'test_accuracy', 'train_loss', 'test_loss'], loc='best')
plt.show()



In [ ]:
model.save("covid_testing.hd5")